In [168]:
from collections import Counter, namedtuple
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [6]:
with open("forum_1000.txt","rb") as f:
    lines = f.readlines()

In [17]:
words_sentences = [w for line in lines for w in line.split()]
len(words_sentences)

57876

In [23]:
dist = Counter(words_sentences)
len(dist.items())

6529

In [72]:
# creating a vocabulary of 1000 most common words skipping the first 100 
vocab = dist.most_common(1100)
vocab = vocab[100:]
vocab = [str(e[0])[2:-1] for e in vocab]
print(len(vocab))
print(vocab[0:7])

1000
['des', 'danke', 'zum', 'gru', 'ihr', 'h', 'war']


In [68]:
def get_feature_vec(text):
    feature_vec = np.zeros(1000)
    for word in text:
        if word in vocab:
            feature_vec[ vocab.index(word) ] += 1
    return feature_vec

In [69]:
text = "hallo zusammen ich bin am berlegen von meinem analog anschluss auf einen ip anschluss zu wechseln gibt es bei der umstellung von einem anaolgen zu einem ip basierten anschluss etwas zu beachten oder funktioniert alles wie gehabt hat jemand erfahrungen ob ich meine alte hardware weiter nutzen kann gr e fabian"
sum(get_feature_vec(text))

112.0

In [ ]:
csv_data = pd.read_csv("full_1000.csv", error_bad_lines=False)

In [84]:
threadIds = csv_data["forumsThread_threadId"]
threadIds = set(threadIds)

In [164]:
# filter threads without answer
threads_to_remove = []
q_a = dict()
# questions = [] 
q_vec = []
X = np.zeros([0,1000])
good_threadIds = []
for tId in threadIds:
    thread_data = csv_data[ csv_data["forumsThread_threadId"]==tId ]
    if sum(thread_data['solution'])==0:
        threads_to_remove.append(tId)
    else:
        q_a[tId] = dict()
        q_a[tId]["questions"] = thread_data[ thread_data['solution']==0 ]['plain_text']
        q_a[tId]["ans"] = thread_data[ thread_data['solution']==1 ]['plain_text']
        for question in q_a[tId]["questions"]:
#             questions.append( (question, tId) )
            feature_vec = get_feature_vec(question)
            q_vec.append( (feature_vec, tId) )
            X = np.concatenate((X, feature_vec.reshape(1,1000)), axis=0)
            good_threadIds.append(tId)

In [165]:
def get_closest_answers(new_question_text):
    feature_vec = get_feature_vec(new_question_text).reshape(1,1000)
    similarities = cosine_similarity(X, feature_vec)
    ind = np.argmax(similarities)
    threadId = good_threadIds[ind]
    return q_a[threadId]["ans"]

In [167]:
print(get_closest_answers(text))

1    Router muss umgestellt werden, Telefone ggf. a...
Name: plain_text, dtype: object


In [171]:
## saving parameters
# with open("naive_LM_params.pkl","wb") as f:
#     pickle.dump([vocab,X],f)